# This is for original code I have removed from the project but I might still need later

# Heston simulation

In [ ]:
def paths(self, S: float, T: float, sigma: float, r: float, v0: float, kappa: float, theta: float, rho: float, M: int, I: int):

    corr_mat = np.zeros((2, 2))
    corr_mat[0, :] = [1.0, rho]
    corr_mat[1, :] = [rho, 1.0]
    cho_mat = np.linalg.cholesky(corr_mat)

    dt = T / M

    ran_num = np.random.standard_normal((2, M + 1, I))

    S_paths = np.zeros_like(ran_num[0])
    v = np.zeros_like(ran_num[0])

    S_paths[0] = S
    v[0] = v0

    for t in range(1, M + 1):
        ran = np.dot(cho_mat, ran_num[:, t, :])

        v[t] = (v[t - 1] +
                kappa * (theta - np.maximum(v[t - 1], 0)) * dt +
                sigma * np.sqrt(np.maximum(v[t - 1], 0)) *
                np.sqrt(dt) * ran[1])
        
        S_paths[t] = S_paths[t - 1] * np.exp((r - 0.5 * v[t]) * dt +
                                np.sqrt(v[t]) * ran[0] * np.sqrt(dt))
        
    return S_paths

    def calculate(self, market_price: float, S: int, K: int, T: float, r: float = .05, initial_guess: float = .2, otype: str = "call", q = .02, **kwargs):

        interval: list[int] = [i/100000 for i in range(0, 200000)]
        
        first = 0
        last = len(interval) - 1
        while first <= last:
            mid = (first + last) // 2
            x = interval[mid]
            #print(f"iv {x}")

            if otype == 'call':
                model_price = self.model.call(S, K, T, x, r, q, **kwargs)
            else:
                model_price = self.model.put(S, K, T, x, r, q, **kwargs)
            print(f"abs(market_price - model_price): {abs(market_price - model_price)}")
            if abs(market_price - model_price) < .001:
                return x
            else:
                if market_price < model_price:
                    last = mid - 1
                else:
                    first = mid + 1
        return False

        iv = initial_guess

        for _ in range(1000):

            if otype == 'call':
                model_price = self.model.call(S, K, T, iv, r, q, **kwargs)
            else:
                model_price = self.model.put(S, K, T, iv, r, q, **kwargs)
            
            if abs(market_price - model_price) < .01:
                return iv

            if market_price - model_price > 0:
                lower_vol = iv
            else:
                upper_vol = iv

            iv = (lower_vol + upper_vol) / 2
            print(f"iv {iv}")

        return False


##### From process option chain py file, these were originaly functions for creating our OptionNode but I replaced them for classes that get passed in a specific Node

In [ ]:
def create_call_option_node(ticker: str, date: str, expiration: str, strike: int):

    #call_chain = SingleUpload().upload_chain(ticker, date)[0]
    chain = pu.cache_chain(ticker, date)
    call_chain = chain[0]
    chain_at_exp = call_chain[expiration]
    
    index: int = np.abs(chain_at_exp['Strike'] - strike).argmin()

    strike: int = chain_at_exp['Strike'][index]
    price: float = chain_at_exp['Price'][index]
    bid: float = chain_at_exp['Bid'][index]
    ask: float = chain_at_exp['Ask'][index]
    iv: float = chain_at_exp['Market Implied Volatility'][index]
    vol: int = chain_at_exp['Volume'][index]
    oi: int = chain_at_exp['Open Interest'][index]

    option_node: OptionNode = OptionNode(strike, price, bid, ask, iv, expiration, vol, oi, 'call')

    return option_node

def create_put_option_node(ticker: str, date: str, expiration: str, strike: int):

    chain = pu.cache_chain(ticker, date)

    put_chain = chain[1]

    #put_chain = SingleUpload().upload_chain(ticker, date)[1]
    chain_at_exp = put_chain[expiration]

    index: int = np.abs(chain_at_exp['Strike'] - strike).argmin()

    strike: int = chain_at_exp['Strike'][index]
    price: float = chain_at_exp['Price'][index]
    bid: float = chain_at_exp['Bid'][index]
    ask: float = chain_at_exp['Ask'][index]
    iv: float = chain_at_exp['Market Implied Volatility'][index]
    vol: int = chain_at_exp['Volume'][index]
    oi: int = chain_at_exp['Open Interest'][index]

    option_node: OptionNode = OptionNode(strike, price, bid, ask, iv, expiration, vol, oi, 'put')

    return option_node

In [ ]:
def process_call_data(ticker: str, date: str) -> OptionGraph:

    """This function creates a data strucutre object representing a list of options for a single expiration across strike"""

    expirations = pu.call_chain_exp(ticker, date)

    option_graph: OptionGraph = OptionGraph()

    for expiration in expirations:

        strikes = pu.call_exp_strikes(ticker, date, expiration)

        for strike in strikes:
            
            option = create_call_option_node(ticker, date, expiration, strike)

            option_graph.add_option(option)

    return option_graph

def process_put_data(ticker: str, date: str) -> OptionGraph:

    """This function creates a data strucutre object representing a graph of options for a single expiration across strike"""

    expirations = pu.put_chain_exp(ticker, date)

    option_graph: OptionGraph = OptionGraph()

    for expiration in expirations:

        strikes = pu.put_exp_strikes(ticker, date, expiration)

        for strike in strikes:
            
            option = create_put_option_node(ticker, date, expiration, strike)

            option_graph.add_option(option)

    return option_graph

### From process_option_chain
##### OptionFactory, now current one uses @staticmethod

In [ ]:
class OptionFactory:

    option_nodes: dict = {
        "european" : EuropeanOptionNode,
        "american" : AmericanOptionNode
    }

    def create_option(self, ticker: str, strike: int, expiration: str, close_date: str = TODAY, option_style: str = "european", option_type: str = "call") -> OptionNode:

        option = self.option_nodes.get(option_style.lower())

        if not option:

            return ValueError(f"Unsupported option type {option_style}")
        
        if option_type.lower() == "call":

            return CreateOption(ticker, close_date, expiration, strike).create_call(option)
        
        if option_type.lower() == "put":

            return CreateOption(ticker, close_date, expiration, strike).create_put(option)
        
        else:

            return ValueError(f"Option type {option_type} is not defined")
        
    def create_option_graph(self, ticker: str, close_date: str = TODAY, option_style: str = "european", option_type: str = "call"):
        
        option = self.option_nodes.get(option_style.lower())

        if not option:

            return ValueError(f"Unsupported option type {option_style}")
        
        if option_type.lower() == "call":

            return CreateOptionGraph(ticker, close_date).create_call_graph(option)
        
        if option_type.lower() == "put":

            return CreateOptionGraph(ticker, close_date).create_put_graph(option)
        
        else:

            return ValueError(f"Option type {option_type} is not defined")

## From option_pricing module
### bsm_model and monte_carlo
##### the two classes were meant to be factory/adapter like patterns where we can pass in a OptionNode into but we will save these for a client interface module

In [ ]:
class BSMCalculator:

    _dict: dict = {"call" : BSMCall(),
                   "put" : BSMPut()}

    def calculate(self, S: float, option_node: OptionNode, bsm_formula: str, r = 0, q = 0):

        formula = self._dict[bsm_formula]
        K: float = option_node.get_strike()
        T: float = option_node.get_dte() / 252
        sigma: float = option_node.get_implied_volatility()

        return formula(S, K, T, sigma, r, q)
    
class MonteCarloCalculator:

    def calculate(self, S: float, option_node: OptionNode, monte_carlo_method, r = 0, M = 100, I = 1000):

        K: float = option_node.get_strike()
        T: float = option_node.get_dte() / 252
        sigma: float = option_node.get_implied_volatility()

        return monte_carlo_method(S, K, M, I, T, sigma, r)

## From greek module

### analytical_greeks.py 
##### We got rid of a adapter like object that is passed in a OptionNode and calculates a greek, we will reimplement this later

In [ ]:
class AnalyticalGreekCalculation:

    # This is our Adapter pattern

    def __init__(self, greek: AnalyticalGreek) -> None:
        self._greek: AnalyticalGreek = greek

    def calculate(self, option_node: OptionNode, spot: int, r: float = 0, q: float = 0) -> float:
        strike: int = option_node.get_strike()
        dte: int = option_node.get_dte()
        sigma: float = option_node.get_implied_volatility()
        otype: str = option_node.otype

        return self._greek.calculate(spot, strike, dte, sigma, r, q, otype)

## Original Sticky delta parameterization

In [ ]:
class StickyDelta:

    # In this class we have a function that takes strikes as the x_axis not deltas
    # We manually calculate atm iv in the parameterize method, lets try to decouple this in the future

    def parameterize(self, implied_volatilities: list[float], strikes: list[float], **kwargs) -> list[float]:

        if "S" in kwargs:
            S = kwargs["S"]
        else:
            return ValueError("Keyword argument must have the parameter S")
        
        if "dte" in kwargs:
            dte = kwargs["dte"]

            if dte > 1:
                dte /= 365

        else:
            return ValueError("Keyword argument must have the parameter dte")

        logic = f"iv > .05 and iv < 2.5 and x <= {S}"

        atm_iv_filter: list[tuple(float, float)] = [(iv, x) for iv, x in zip(implied_volatilities, strikes) if eval(logic)]

        atm_iv: float = atm_iv_filter[-1][0]

        filtered_ivs_strikes: list[tuple(float, float)] = [(iv, np.log(x / S) / (atm_iv * np.sqrt(dte))) for iv, x in zip(implied_volatilities, strikes)]# if eval(func)]
        ivs, strikes = zip(*filtered_ivs_strikes)

        return ivs, strikes

## This function is used to calculate the list of a date range of historical stock prices, I opted out of using this function and decided to use the datetimes from the yfinance lib data

# 3/9/25 we got rid of module payoff_diagrams but it is below

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from .positions import Position, PositionNode
from .position_calc import ChangeCalculator
from calc_engine.option_pricing import analytical_solutions as an

class PayoffDiagram:

    def __init__(self, payoff_diagram) -> None:
        self.payoff_diagram = payoff_diagram

    def plot(self, title: bool = False, xlabel: bool = False, ylabel: bool = False):

        y = self.payoff_diagram.get_payoff()
        x = self.payoff_diagram.get_change_variable()

        plt.figure()
        plt.plot(x, y)
        plt.show()

    def plot_exp(self, title: bool = False, xlabel: bool = False, ylabel: bool = False):

        y = self.payoff_diagram.get_payoff()
        y_exp = self.payoff_diagram.get_payoff_exp()
        x = self.payoff_diagram.get_change_variable()

        print(f"y expiration: {y_exp}\n")
        print(f"y: {y}\n")

        plt.figure()
        plt.plot(x, y, 'r')
        plt.plot(x, y_exp, 'b')
        plt.show()

    def plot_expiration(self):
        pass

class PayoffSpot:

    def __init__(self) -> None:
        self.payoff: list | np.ndarray = None 
        self.change_variable: list | np.ndarray = None
        self.payoff_exp: list | np.ndarray = None

    def get_payoff(self):
        return self.payoff
    
    def get_change_variable(self):
        return self.change_variable
    
    def get_payoff_exp(self):
        return self.payoff_exp

    def set_payoff(self, change_variable: np.ndarray | list, spot: float, strike: int, expiration: float, sigma: float, r: float = 0, q: float = 0, otype: str = "call", exposure: str = "long"):

        calculator = ChangeCalculator()

        self.payoff = calculator.calculate_change("spot", change_variable, spot, strike, expiration, sigma, r, q, otype, exposure)

        self.change_variable = change_variable

        #payoff_diagram = PayoffDiagram().plot(change_variable, payoff)

    def set_position_payoff(self, position: Position, change_variable: list | np.ndarray):
        
        calculator = ChangeCalculator()

        position_payoff = calculator.calculate_position(position, "spot", change_variable)

        position_payoff_exp = calculator.calculate_position_expiration(position, "spot", change_variable)

        self.payoff = position_payoff

        self.change_variable = change_variable

        self.payoff_exp = position_payoff_exp

        #payoff_diagram = PayoffDiagram().plot(change_variable, position_payoff)

class PayoffSigma:

    def __init__(self) -> None:
        self.payoff: list | np.ndarray = None 
        self.change_variable: list | np.ndarray = None
        self.payoff_exp: list | np.ndarray = None

    def get_payoff(self):
        return self.payoff
    
    def get_change_variable(self):
        return self.change_variable
    
    def get_payoff_exp(self):
        return self.payoff_exp
    
    def set_payoff(self, change_variable: np.ndarray | list, spot: float, strike: int, expiration: float, sigma: float, r: float = 0, q: float = 0, otype: str = "call", exposure: str = "long"):

        calculator = ChangeCalculator()

        self.payoff = calculator.calculate_change("sigma", change_variable, spot, strike, expiration, sigma, r, q, otype, exposure)

        self.change_variable = change_variable

    def set_position_payoff(self, position: Position, change_variable: list | np.ndarray):
        
        calculator = ChangeCalculator()

        position_payoff = calculator.calculate_position(position, "sigma", change_variable)

        position_payoff_exp = calculator.calculate_position_expiration(position, "sigma", change_variable)

        self.payoff = position_payoff

        self.change_variable = change_variable

        self.payoff_exp = position_payoff_exp

class PayoffTime:

    def __init__(self) -> None:
        self.payoff: list | np.ndarray = None 
        self.change_variable: list | np.ndarray = None
        self.payoff_exp: list | np.ndarray = None

    def get_payoff(self):
        return self.payoff
    
    def get_change_variable(self):
        return self.change_variable
    
    def get_payoff_exp(self):
        return self.payoff_exp
    
    def set_payoff(self, change_variable: np.ndarray | list, spot: float, strike: int, expiration: float, sigma: float, r: float = 0, q: float = 0, otype: str = "call", exposure: str = "long"):

        calculator = ChangeCalculator()

        self.payoff = calculator.calculate_change("time", change_variable, spot, strike, expiration, sigma, r, q, otype, exposure)

        self.change_variable = change_variable

    def set_position_payoff(self, position: Position, change_variable: list | np.ndarray):
        
        calculator = ChangeCalculator()

        position_payoff = calculator.calculate_position(position, "time", change_variable)

        position_payoff_exp = calculator.calculate_position_expiration(position, "time", change_variable)

        self.payoff = position_payoff

        self.change_variable = change_variable

        self.payoff_exp = position_payoff_exp